In [ ]:
!pip install networkx

In [ ]:
!pip install scipy 

In [ ]:
!pip install matplotlib

In [112]:
import networkx as nx
import random
import matplotlib.pyplot as plt
from collections import deque
import concurrent.futures

G = nx.erdos_renyi_graph(1000, 0.0001,directed=True)

for node in G.nodes():
    G.nodes[node]['weight'] = random.uniform(0, 1)

for u, v in G.edges():
    G.edges[u, v]['weights'] = (
        random.uniform(0, 1),
        random.uniform(0, 1),
        random.uniform(0, 1)
    )

In [ ]:
#In ra số cạnh của G
print('Số cạnh: ',G.number_of_edges())
print('Các đỉnh cạnh 1:')
for neighbor in G.neighbors(0):
    print(neighbor)

Số cạnh:  99


## Hàm tính ảnh hưởng

In [107]:
def influence_spread(G, seed_set):
    """
    G: Đồ thị với trọng số cạnh là bộ 3 số tương ứng 3 chủ đề.
    seed_set: List các tuple (node, topic) – tập node ban đầu bị ảnh hưởng theo chủ đề.
    
    Trả về: dict {topic: set(node bị ảnh hưởng theo topic)}
    """
    activated = dict()  # node -> topic bị ảnh hưởng đầu tiên
    queue = deque(seed_set)

    for node, topic in seed_set:
        activated[node] = topic

    while queue:
        current_node, topic = queue.popleft()

        for neighbor in G.neighbors(current_node):
            if neighbor in activated:
                continue  # đã bị ảnh hưởng rồi

            weight_tuple = G.edges[current_node, neighbor].get('weights', (0, 0, 0))
            influence_prob = weight_tuple[topic]

            if random.random() <= influence_prob:
                activated[neighbor] = topic
                queue.append((neighbor, topic))

    topic_influence = {0: set(), 1: set(), 2: set()}
    for node, topic in activated.items():
        topic_influence[topic].add(node)

    return topic_influence

In [108]:
def monte_carlo_parallel(G, seed_set, num_simulations=100):
    from concurrent.futures import ThreadPoolExecutor

    def single_simulation(_):
        return len(set.union(*influence_spread(G, seed_set).values()))

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(single_simulation, range(num_simulations)))

    return sum(results) / num_simulations


In [111]:
avg_total = monte_carlo_parallel(G, seed_set=[(0, 1)], num_simulations=1000)
print(f"Trung bình số node bị ảnh hưởng: {avg_total:.2f}")

Trung bình số node bị ảnh hưởng: 1.00
